In [2]:
import os
import random
import time
import wandb
import numpy as np
import gymnasium as gym
import flax.linen as nn
from flax.training.train_state import TrainState
from flax.training.common_utils import onehot
import jax.numpy as jnp
import jax
import optax

from torch.utils.tensorboard import SummaryWriter

In [3]:
def make_env(env_id, seed, idx, capture_video, run_name):
    def thunk():
        if capture_video and idx == 0:
            env = gym.make(env_id,render_mode="rgb_array")
            env = gym.wrappers.RecordVideo(env,f"videos/{run_name}")
        else:
            env = gym.make(env_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        env.action_space.seed(seed)
        return env
    return thunk

In [4]:
class MLPPolicy(nn.Module):
    action_dims: int

    @nn.compact
    def __call__(self, input: jnp.ndarray):
        x = nn.Dense(16)(input)
        x = nn.relu(x)
        x = nn.Dense(16)(x)
        x = nn.relu(x)
        x = nn.Dense(self.action_dims)(x)
        probs = nn.softmax(x)
        log_probs = nn.log_softmax(x)

        return probs,log_probs

In [5]:
def train(env_id,gamma,episodes,max_termination,seed,num_envs,learning_rate):
    run_name = f"{env_id}__{seed}_{int(time.time())}"
    wandb.init(
        project="reinforce-classic-control-benchmark",
        config={
            "env_id":env_id,
            "gamma":gamma,
            "epsisodes":episodes,
            "max_termination":max_termination,
            "seed":seed,
            "num_envs":num_envs,
            "lr":learning_rate,
        },
        sync_tensorboard=True,
        monitor_gym=True,
        name=run_name
    )
    writer = SummaryWriter(f"runs/{run_name}")

    random.seed(seed)
    np.random.seed(seed=seed)
    key = jax.random.PRNGKey(seed)

    env = make_env(env_id, seed, 0, True, run_name)()

    obs, _ = env.reset(seed=seed)
    action_dims = env.action_space.n

    policy = MLPPolicy(action_dims=action_dims)

    policy_state = TrainState.create(
        apply_fn=policy.apply,
        params=policy.init(key,obs),
        tx=optax.adam(learning_rate=learning_rate)
    )

    policy.apply = jax.jit(policy.apply)

    @jax.jit
    def update(policy_state,observations,actions,rewards):
        def loss_fn(params):
            _,logprobs = policy.apply(params,observations)
            onehot_actions = onehot(actions,num_classes=logprobs.shape[-1]).reshape(logprobs.shape)
            selected_action_logprobs = jnp.sum(onehot_actions * logprobs, axis=-1)
            loss_value = -jnp.sum(selected_action_logprobs * rewards)
            return loss_value

        loss_value,grads = jax.value_and_grad(loss_fn)(policy_state.params)
        policy_state = policy_state.apply_gradients(grads=grads)
        return loss_value,policy_state

    for episode in range(episodes):
        rewards = []
        # logprobs = []
        actions = []
        observations = []

        done = False
        obs, _ = env.reset(seed=seed)
        for _ in range(max_termination):
            observations.append(obs)
            prob,_ = policy.apply(policy_state.params,jnp.array(obs)[None,...])
            prob = jax.device_get(prob)[0]
            action = np.random.choice(action_dims,p=prob)
            obs, reward, done, _, _ = env.step(int(action))
            actions.append(action)
            rewards.append(reward)
            if done:
                break

        returns = []
        R = 0
        for r in reversed(rewards):
            R = r + gamma * R
            returns.insert(0,R)

        returns = jnp.array(returns)
        eps = np.finfo(np.float32).eps.item()
        returns = (returns - returns.mean())/(returns.std() + eps)
        actions = jnp.array(actions)
        observations = jnp.array(observations)

        loss, policy_state = update(
            policy_state,
            observations,
            actions,
            returns
        )
        print(f"Episode:{episode}   Loss:{loss}  Reward:{sum(rewards)} Episode Lenght:{len(rewards)}")
        writer.add_scalar("loss",jax.device_get(loss),episode)
        writer.add_scalar("rewards",sum(rewards),episode)

    writer.close()
    env.close()

    return policy_state


In [10]:
envs = {"cartPole":"CartPole-v1","acrobot":"Acrobot-v1","mountainCar":"MountainCar-v0"}
gamma = 0.9
episodes = 4000
max_termination = 1000
seed = 0
num_envs = 1
learning_rate = 0.001

In [11]:
train(envs["mountainCar"],gamma,episodes,max_termination,seed,num_envs,learning_rate)

global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▄▅▆▃▅▆▆▄▅▄▃▃▄▃▅▅▃▆▆▅▄▆▄▅▅▃▄▄▇▇▆▆▅▃▅▁█▃▃▅
rewards,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
global_step,3999
loss,1.16523
rewards,-1000.0


wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


Moviepy - Building video /notebooks/rl-algos/videos/MountainCar-v0__0_1699956346/rl-video-episode-0.mp4.
Moviepy - Writing video /notebooks/rl-algos/videos/MountainCar-v0__0_1699956346/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/MountainCar-v0__0_1699956346/rl-video-episode-0.mp4
Episode:0   Loss:-1.880661964416504  Reward:-1000.0 Episode Lenght:1000
Moviepy - Building video /notebooks/rl-algos/videos/MountainCar-v0__0_1699956346/rl-video-episode-1.mp4.
Moviepy - Writing video /notebooks/rl-algos/videos/MountainCar-v0__0_1699956346/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/MountainCar-v0__0_1699956346/rl-video-episode-1.mp4
Episode:1   Loss:3.0718746185302734  Reward:-1000.0 Episode Lenght:1000
Episode:2   Loss:-3.5280046463012695  Reward:-1000.0 Episode Lenght:1000
Episode:3   Loss:11.557256698608398  Reward:-1000.0 Episode Lenght:1000
Episode:4   Loss:-1.5823020935058594  Reward:-1000.0 Episode Lenght:1000
Episode:5   Loss:0.5212497711181641  Reward:-1000.0 Episode Lenght:1000
Episode:6   Loss:4.905426025390625  Reward:-1000.0 Episode Lenght:1000
Episode:7   Loss:0.02987957000732422  Reward:-1000.0 Episode Lenght:1000
Moviepy - Building video /notebooks/rl-algos/videos/MountainCar-v0__0_1699956346/rl-video-episode-8.mp4.
Moviepy - Writing video /notebooks/rl-algos/videos/MountainCar-v0__0_1699956346/rl-video-episode-8.mp4



...

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/MountainCar-v0__0_1699956346/rl-video-episode-1000.mp4
Episode:1000   Loss:-0.9080810546875  Reward:-1000.0 Episode Lenght:1000
Episode:1001   Loss:10.299215316772461  Reward:-1000.0 Episode Lenght:1000
Episode:1002   Loss:10.204967498779297  Reward:-1000.0 Episode Lenght:1000
Episode:1003   Loss:-3.2799606323242188  Reward:-1000.0 Episode Lenght:1000
Episode:1004   Loss:-22.775943756103516  Reward:-1000.0 Episode Lenght:1000
Episode:1005   Loss:7.537966728210449  Reward:-1000.0 Episode Lenght:1000
Episode:1006   Loss:-0.7899265289306641  Reward:-1000.0 Episode Lenght:1000
Episode:1007   Loss:-4.808506011962891  Reward:-1000.0 Episode Lenght:1000
Episode:1008   Loss:-0.9596967697143555  Reward:-1000.0 Episode Lenght:1000
Episode:1009   Loss:-20.238126754760742  Reward:-1000.0 Episode Lenght:1000
Episode:1010   Loss:-10.253435134887695  Reward:-1000.0 Episode Lenght:1000
Episode:1011   Loss:4.573627471923828  Reward:-1000

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/MountainCar-v0__0_1699956346/rl-video-episode-2000.mp4
Episode:2000   Loss:12.280218124389648  Reward:-1000.0 Episode Lenght:1000
Episode:2001   Loss:-5.962685585021973  Reward:-1000.0 Episode Lenght:1000
Episode:2002   Loss:24.006797790527344  Reward:-1000.0 Episode Lenght:1000
Episode:2003   Loss:19.620420455932617  Reward:-1000.0 Episode Lenght:1000
Episode:2004   Loss:18.79877471923828  Reward:-1000.0 Episode Lenght:1000
Episode:2005   Loss:-1.6074481010437012  Reward:-1000.0 Episode Lenght:1000
Episode:2006   Loss:-24.88214874267578  Reward:-1000.0 Episode Lenght:1000
Episode:2007   Loss:47.646141052246094  Reward:-1000.0 Episode Lenght:1000
Episode:2008   Loss:-4.525152206420898  Reward:-1000.0 Episode Lenght:1000
Episode:2009   Loss:2.6621837615966797  Reward:-1000.0 Episode Lenght:1000
Episode:2010   Loss:26.06179428100586  Reward:-1000.0 Episode Lenght:1000
Episode:2011   Loss:9.930980682373047  Reward:-1000.0 E

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/MountainCar-v0__0_1699956346/rl-video-episode-3000.mp4
Episode:3000   Loss:-14.086091041564941  Reward:-1000.0 Episode Lenght:1000
Episode:3001   Loss:-2.6843137741088867  Reward:-1000.0 Episode Lenght:1000
Episode:3002   Loss:-23.403575897216797  Reward:-1000.0 Episode Lenght:1000
Episode:3003   Loss:20.798912048339844  Reward:-1000.0 Episode Lenght:1000
Episode:3004   Loss:-22.43109703063965  Reward:-1000.0 Episode Lenght:1000
Episode:3005   Loss:19.44402503967285  Reward:-1000.0 Episode Lenght:1000
Episode:3006   Loss:4.826547622680664  Reward:-1000.0 Episode Lenght:1000
Episode:3007   Loss:-5.947260856628418  Reward:-1000.0 Episode Lenght:1000
Episode:3008   Loss:-12.108234405517578  Reward:-1000.0 Episode Lenght:1000
Episode:3009   Loss:-24.606449127197266  Reward:-1000.0 Episode Lenght:1000
Episode:3010   Loss:9.00093936920166  Reward:-1000.0 Episode Lenght:1000
Episode:3011   Loss:-19.485910415649414  Reward:-1000

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/MountainCar-v0__0_1699956346/rl-video-episode-4000.mp4


TrainState(step=Array(4000, dtype=int32, weak_type=True), apply_fn=<bound method Module.apply of MLPPolicy(
    # attributes
    action_dims = 3
)>, params=FrozenDict({
    params: {
        Dense_0: {
            bias: Array([-0.04243626, -0.04255573, -0.04262176,  0.05070423, -0.04024125,
                   -0.04434826, -0.05914595, -0.02557674, -0.04410316, -0.1105606 ,
                   -0.02047009,  0.01735428, -0.03230472, -0.07481626, -0.03225683,
                   -0.02252995], dtype=float32),
            kernel: Array([[-0.44260874, -1.5601882 , -1.1346977 ,  0.9341963 ,  0.02998589,
                     0.81851006,  0.04668314,  0.14245598, -1.1609331 , -0.2739778 ,
                    -0.68549204,  0.27432388,  0.9784788 , -0.766398  , -0.73717576,
                    -0.6822712 ],
                   [-0.24176899, -0.4008417 ,  0.19514215, -0.20621327,  0.94435525,
                     1.0735552 , -1.5468733 ,  0.42790997, -0.11145167,  0.8676161 ,
                    -0.1